In [264]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [266]:
# read in dataset
dfWalk = pd.read_csv('/Users/austincoffelt/Downloads/Rent-Walk-local/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv')
dfWalk.head()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,7825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,297836.0831
1,2,4.811300e+11,4.811300e+11,48,113,7825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,484945.1466
2,3,4.811300e+11,4.811300e+11,48,113,7825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,106705.9281
3,4,4.811300e+11,4.811300e+11,48,113,7824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,481828.4303
4,5,4.811300e+11,4.811300e+11,48,113,7824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,687684.7752


In [267]:
# change state, county, tract, and blk group to strings
dfWalk.loc[:, ['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE']] = dfWalk.loc[:, ['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE']].astype('str')

In [268]:
# add 0s to the front to create the geoIDs
for i in range(len(dfWalk)):
    dfWalk.loc[i, 'STATEFP'] = dfWalk.loc[i, 'STATEFP'].zfill(2)
    dfWalk.loc[i, 'COUNTYFP'] = dfWalk.loc[i, 'COUNTYFP'].zfill(3)
    dfWalk.loc[i, 'TRACTCE'] = dfWalk.loc[i, 'TRACTCE'].zfill(6)
    
dfWalk.head()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,4.811300e+11,4.811300e+11,48,113,007825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.184697,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,297836.0831
1,2,4.811300e+11,4.811300e+11,48,113,007825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,484945.1466
2,3,4.811300e+11,4.811300e+11,48,113,007825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,106705.9281
3,4,4.811300e+11,4.811300e+11,48,113,007824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,481828.4303
4,5,4.811300e+11,4.811300e+11,48,113,007824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,687684.7752


In [269]:
# create realGeoID for merging
dfWalk['realGEOID'] = '1500000US' + dfWalk['STATEFP'] + dfWalk['COUNTYFP'] + dfWalk['TRACTCE'] + dfWalk['BLKGRPCE']
dfWalk.head()

,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area,realGEOID
0,1,4.811300e+11,4.811300e+11,48,113,007825,4,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000476,0.137707,6,14,15,17,14.000000,3110.360820,297836.0831,1500000US481130078254
1,2,4.811300e+11,4.811300e+11,48,113,007825,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000801,0.231868,3,10,12,14,10.833333,3519.469110,484945.1466,1500000US481130078252
2,3,4.811300e+11,4.811300e+11,48,113,007825,3,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,106705.9281,1500000US481130078253
3,4,4.811300e+11,4.811300e+11,48,113,007824,1,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,481828.4303,1500000US481130078241
4,5,4.811300e+11,4.811300e+11,48,113,007824,2,206.0,"Dallas-Fort Worth, TX-OK",19100.0,...,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,687684.7752,1500000US481130078242


In [270]:
# create intercept constant
dfWalk['Intercept'] = 1

In [273]:
# select and rename the most important columns of our dataframe
imptCols = ['realGEOID', 'Intercept', 'NatWalkInd', 'STATEFP', 'P_WrkAge', 'Pct_AO2p',
            'R_HiWageWk', 'D1B', 'D1D', 'D2A_JPHH', 'D4E', 'D5AR', 'D5BR', 'D5CRI']
dfWalk = dfWalk.loc[:, imptCols]
dfWalk = dfWalk.rename({'NatWalkInd':'Walk_Index',
                        'STATEFP':'state',
                        'P_WrkAge':'percentage_work_age',
                        'Pct_AO2p':'percent_two_car',
                        'R_HiWageWk':'count_high_wage_workers',
                        'D1B':'population_density',
                        'D1D':'employent_housing_density',
                        'D2A_JPHH':'jobs_per_household',
                        'D4E':'transit_frequency',
                        'D5AR':'jobs_within_45_minutes_auto',
                        'D5BR':'jobs_within_45_minutes_transit',
                        'D5CRI':'regional_centrality'}, axis = 1)
dfWalk.head()

,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,jobs_within_45_minutes_auto,jobs_within_45_minutes_transit,regional_centrality
0,1500000US481130078254,1,14.000000,48,0.549,0.744681,191,16.332625,7.147222,0.156028,0.003602,433601,135362,0.785893
1,1500000US481130078252,1,10.833333,48,0.466,0.589242,212,5.955666,3.640506,0.061125,0.006099,386504,236885,0.700531
2,1500000US481130078253,1,8.333333,48,0.811,0.507599,138,27.951553,13.843035,0.000000,0.004071,404573,230587,0.733281
3,1500000US481130078241,1,15.666667,48,0.638,0.888021,302,7.592767,5.350213,0.658854,0.007378,423099,168433,0.766859
4,1500000US481130078242,1,10.166667,48,0.506,0.790087,404,6.373413,2.521128,0.093294,0.007036,335700,120826,0.608450


In [274]:
# change numbers to States
dfWalk.loc[dfWalk['state'] == '01', 'state'] = 'Alabama' 
dfWalk.loc[dfWalk['state'] == '02', 'state'] = 'Alaska'
dfWalk.loc[dfWalk['state'] == '04', 'state'] = 'Arizona'
dfWalk.loc[dfWalk['state'] == '05', 'state'] = 'Arkansas'
dfWalk.loc[dfWalk['state'] == '06', 'state'] = 'California'
dfWalk.loc[dfWalk['state'] == '08', 'state'] = 'Colorado'
dfWalk.loc[dfWalk['state'] == '09', 'state'] = 'Connecticut'
dfWalk.loc[dfWalk['state'] == '10', 'state'] = 'Deleware'
dfWalk.loc[dfWalk['state'] == '11', 'state'] = 'District of Columbia'
dfWalk.loc[dfWalk['state'] == '12', 'state'] = 'Florida'
dfWalk.loc[dfWalk['state'] == '13', 'state'] = 'Georgia'
dfWalk.loc[dfWalk['state'] == '15', 'state'] = 'Hawaii'
dfWalk.loc[dfWalk['state'] == '16', 'state'] = 'Idaho'
dfWalk.loc[dfWalk['state'] == '17', 'state'] = 'Illinois'
dfWalk.loc[dfWalk['state'] == '18', 'state'] = 'Indiana'
dfWalk.loc[dfWalk['state'] == '19', 'state'] = 'Iowa'
dfWalk.loc[dfWalk['state'] == '20', 'state'] = 'Kansas'
dfWalk.loc[dfWalk['state'] == '21', 'state'] = 'Kentucky'
dfWalk.loc[dfWalk['state'] == '22', 'state'] = 'Louisiana'
dfWalk.loc[dfWalk['state'] == '23', 'state'] = 'Maine'
dfWalk.loc[dfWalk['state'] == '24', 'state'] = 'Maryland'
dfWalk.loc[dfWalk['state'] == '25', 'state'] = 'Massachusetts'
dfWalk.loc[dfWalk['state'] == '26', 'state'] = 'Michigan'
dfWalk.loc[dfWalk['state'] == '27', 'state'] = 'Minnesota'
dfWalk.loc[dfWalk['state'] == '28', 'state'] = 'Mississippi'
dfWalk.loc[dfWalk['state'] == '29', 'state'] = 'Missouri'
dfWalk.loc[dfWalk['state'] == '30', 'state'] = 'Montana'
dfWalk.loc[dfWalk['state'] == '31', 'state'] = 'Nebraska'
dfWalk.loc[dfWalk['state'] == '32', 'state'] = 'Nevada'
dfWalk.loc[dfWalk['state'] == '33', 'state'] = 'New Hampshire'
dfWalk.loc[dfWalk['state'] == '34', 'state'] = 'New Jersey'
dfWalk.loc[dfWalk['state'] == '35', 'state'] = 'New Mexico'
dfWalk.loc[dfWalk['state'] == '36', 'state'] = 'New York'
dfWalk.loc[dfWalk['state'] == '37', 'state'] = 'North Carolina'
dfWalk.loc[dfWalk['state'] == '38', 'state'] = 'North Dakota'
dfWalk.loc[dfWalk['state'] == '39', 'state'] = 'Ohio'
dfWalk.loc[dfWalk['state'] == '40', 'state'] = 'Oklahoma'
dfWalk.loc[dfWalk['state'] == '41', 'state'] = 'Oregon'
dfWalk.loc[dfWalk['state'] == '42', 'state'] = 'Pennsylvania'
dfWalk.loc[dfWalk['state'] == '44', 'state'] = 'Rhode Island'
dfWalk.loc[dfWalk['state'] == '45', 'state'] = 'South Carolian'
dfWalk.loc[dfWalk['state'] == '46', 'state'] = 'South Dakota'
dfWalk.loc[dfWalk['state'] == '47', 'state'] = 'Tennessee'
dfWalk.loc[dfWalk['state'] == '48', 'state'] = 'Texas'
dfWalk.loc[dfWalk['state'] == '49', 'state'] = 'Utah'
dfWalk.loc[dfWalk['state'] == '50', 'state'] = 'Vermont'
dfWalk.loc[dfWalk['state'] == '51', 'state'] = 'Virginia'
dfWalk.loc[dfWalk['state'] == '53', 'state'] = 'Washington'
dfWalk.loc[dfWalk['state'] == '54', 'state'] = 'West Virginia'
dfWalk.loc[dfWalk['state'] == '55', 'state'] = 'Wisconsin'
dfWalk.loc[dfWalk['state'] == '56', 'state'] = 'Wyoming'
dfWalk.head()

,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,jobs_within_45_minutes_auto,jobs_within_45_minutes_transit,regional_centrality
0,1500000US481130078254,1,14.000000,Texas,0.549,0.744681,191,16.332625,7.147222,0.156028,0.003602,433601,135362,0.785893
1,1500000US481130078252,1,10.833333,Texas,0.466,0.589242,212,5.955666,3.640506,0.061125,0.006099,386504,236885,0.700531
2,1500000US481130078253,1,8.333333,Texas,0.811,0.507599,138,27.951553,13.843035,0.000000,0.004071,404573,230587,0.733281
3,1500000US481130078241,1,15.666667,Texas,0.638,0.888021,302,7.592767,5.350213,0.658854,0.007378,423099,168433,0.766859
4,1500000US481130078242,1,10.166667,Texas,0.506,0.790087,404,6.373413,2.521128,0.093294,0.007036,335700,120826,0.608450


In [275]:
# get race and population
temp = pd.read_csv('/Users/austincoffelt/Downloads/Rent-Walk-local/Race_2018/Race_2018.csv', header = 1)
temp['percent_non_white'] = 1 - (temp['Estimate!!Total!!White alone'] / temp['Estimate!!Total'])
temp = temp.rename({'Estimate!!Total': 'Population'}, axis = 1)
temp = temp[['Geography', 'Population', 'percent_non_white']]
temp.head()

,Geography,Population,percent_non_white
0,1500000US010010201001,636,0.141509
1,1500000US010010201002,1287,0.174048
2,1500000US010010202001,810,0.676543
3,1500000US010010202002,1218,0.542693
4,1500000US010010203001,2641,0.340401


In [276]:
# merge dfWalk with rent data on GEOID
dfWalk = dfWalk.merge(temp, how = 'inner', left_on = 'realGEOID', right_on = 'Geography')
dfWalk = dfWalk.drop('Geography', axis = 1)
print(dfWalk.shape)
dfWalk.head()

(220333, 16)


,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,jobs_within_45_minutes_auto,jobs_within_45_minutes_transit,regional_centrality,Population,percent_non_white
0,1500000US481130078254,1,14.000000,Texas,0.549,0.744681,191,16.332625,7.147222,0.156028,0.003602,433601,135362,0.785893,1202,0.400998
1,1500000US481130078252,1,10.833333,Texas,0.466,0.589242,212,5.955666,3.640506,0.061125,0.006099,386504,236885,0.700531,710,0.119718
2,1500000US481130078253,1,8.333333,Texas,0.811,0.507599,138,27.951553,13.843035,0.000000,0.004071,404573,230587,0.733281,737,0.770692
3,1500000US481130078241,1,15.666667,Texas,0.638,0.888021,302,7.592767,5.350213,0.658854,0.007378,423099,168433,0.766859,904,0.095133
4,1500000US481130078242,1,10.166667,Texas,0.506,0.790087,404,6.373413,2.521128,0.093294,0.007036,335700,120826,0.608450,948,0.047468


In [277]:
# get median rent
temp = pd.read_csv('/Users/austincoffelt/Downloads/Rent-Walk-local/Rent_2018/Rent_2018.csv', header = 1)
temp = temp.rename({'Estimate!!Median contract rent': 'Median_Contract_Rent'}, axis = 1)
temp = temp[['Geography', 'Median_Contract_Rent']]

# remove observations without numbers
temp = temp[temp['Median_Contract_Rent'] != '-']
temp.head()

,Geography,Median_Contract_Rent
0,1500000US010010201001,607
1,1500000US010010201002,532
2,1500000US010010202001,404
3,1500000US010010202002,646
4,1500000US010010203001,685


In [278]:
# merge dfWalk with rent data on GEOID
dfWalk = dfWalk.merge(temp, how = 'inner', left_on = 'realGEOID', right_on = 'Geography')
dfWalk = dfWalk.drop('Geography', axis = 1)
print(dfWalk.shape)
dfWalk.head()

(176904, 17)


,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,jobs_within_45_minutes_auto,jobs_within_45_minutes_transit,regional_centrality,Population,percent_non_white,Median_Contract_Rent
0,1500000US481130078254,1,14.000000,Texas,0.549,0.744681,191,16.332625,7.147222,0.156028,0.003602,433601,135362,0.785893,1202,0.400998,838
1,1500000US481130078253,1,8.333333,Texas,0.811,0.507599,138,27.951553,13.843035,0.000000,0.004071,404573,230587,0.733281,737,0.770692,707
2,1500000US481130078271,1,6.833333,Texas,0.588,0.227364,206,26.357776,11.028441,0.006036,0.002246,402287,138562,0.729137,1336,0.561377,859
3,1500000US481130093012,1,8.000000,Texas,0.644,0.915761,137,7.680394,2.252783,0.002717,0.002596,263813,8873,0.478156,1541,0.143413,581
4,1500000US481130011022,1,13.166667,Texas,0.690,0.552941,302,9.110446,4.797439,0.203922,0.009142,372503,275466,0.675155,583,0.102916,1375


In [279]:
# get average household size
temp = pd.read_csv('/Users/austincoffelt/Downloads/Rent-Walk-local/HH_Size_2019/HH_Size_2019.csv', header = 1)
temp = temp.rename({'Estimate!!Average household size --!!Total:!!Renter occupied': 'avg_HH_size_renters'}, axis = 1)
temp = temp[['Geography', 'avg_HH_size_renters']]

# remove observations without values
temp = temp[temp['avg_HH_size_renters'] != '-']
temp.head()

,Geography,avg_HH_size_renters
0,1500000US010010201001,1.78
1,1500000US010010201002,3.40
2,1500000US010010202001,2.42
3,1500000US010010202002,2.14
4,1500000US010010203001,2.49


In [280]:
# merge dfWalk with household data on GEOID
dfWalk = dfWalk.merge(temp, how = 'inner', left_on = 'realGEOID', right_on = 'Geography')
dfWalk = dfWalk.drop('Geography', axis = 1)
print(dfWalk.shape)
dfWalk.head()

(175841, 18)


,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,jobs_within_45_minutes_auto,jobs_within_45_minutes_transit,regional_centrality,Population,percent_non_white,Median_Contract_Rent,avg_HH_size_renters
0,1500000US481130078254,1,14.000000,Texas,0.549,0.744681,191,16.332625,7.147222,0.156028,0.003602,433601,135362,0.785893,1202,0.400998,838,2.55
1,1500000US481130078253,1,8.333333,Texas,0.811,0.507599,138,27.951553,13.843035,0.000000,0.004071,404573,230587,0.733281,737,0.770692,707,2.05
2,1500000US481130078271,1,6.833333,Texas,0.588,0.227364,206,26.357776,11.028441,0.006036,0.002246,402287,138562,0.729137,1336,0.561377,859,2.72
3,1500000US481130093012,1,8.000000,Texas,0.644,0.915761,137,7.680394,2.252783,0.002717,0.002596,263813,8873,0.478156,1541,0.143413,581,4.81
4,1500000US481130011022,1,13.166667,Texas,0.690,0.552941,302,9.110446,4.797439,0.203922,0.009142,372503,275466,0.675155,583,0.102916,1375,1.88


In [281]:
# get median household income data
temp = pd.read_csv('/Users/austincoffelt/Downloads/Rent-Walk-Local/HH_Income_2018/HH_Income_2018.csv', header = 1)
temp = temp.rename({'Estimate!!Median household income in the past 12 months (in 2018 inflation-adjusted dollars)': 'Median_Household_Income'}, axis = 1)
temp = temp[['Geography', 'Median_Household_Income']]

# remove observations without numbers
temp = temp[temp['Median_Household_Income'] != '-']
temp.head()

,Geography,Median_Household_Income
0,1500000US010010201001,26579
1,1500000US010010201002,82750
2,1500000US010010202001,27500
3,1500000US010010202002,49276
4,1500000US010010203001,58235


In [282]:
# merge dfWalk with median household income data on GEOID
dfWalk = dfWalk.merge(temp, how = 'inner', left_on = 'realGEOID', right_on = 'Geography')
dfWalk = dfWalk.drop('Geography', axis = 1)
print(dfWalk.shape)
dfWalk.head()

(171847, 19)


,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,jobs_within_45_minutes_auto,jobs_within_45_minutes_transit,regional_centrality,Population,percent_non_white,Median_Contract_Rent,avg_HH_size_renters,Median_Household_Income
0,1500000US481130078254,1,14.000000,Texas,0.549,0.744681,191,16.332625,7.147222,0.156028,0.003602,433601,135362,0.785893,1202,0.400998,838,2.55,54154
1,1500000US481130078253,1,8.333333,Texas,0.811,0.507599,138,27.951553,13.843035,0.000000,0.004071,404573,230587,0.733281,737,0.770692,707,2.05,33996
2,1500000US481130078271,1,6.833333,Texas,0.588,0.227364,206,26.357776,11.028441,0.006036,0.002246,402287,138562,0.729137,1336,0.561377,859,2.72,31213
3,1500000US481130093012,1,8.000000,Texas,0.644,0.915761,137,7.680394,2.252783,0.002717,0.002596,263813,8873,0.478156,1541,0.143413,581,4.81,53191
4,1500000US481130011022,1,13.166667,Texas,0.690,0.552941,302,9.110446,4.797439,0.203922,0.009142,372503,275466,0.675155,583,0.102916,1375,1.88,139875


In [283]:
# get median rooms
temp = pd.read_csv('/Users/austincoffelt/Downloads/Rent-Walk-Local/Median_Rooms_2018/Median_Rooms_2018.csv', header = 1)
temp = temp.rename({'Estimate!!Median number of rooms --!!Renter occupied': 'Median_Num_Rooms'}, axis = 1)
temp = temp[['Geography', 'Median_Num_Rooms']]

# remove observations without numbers
temp = temp[temp['Median_Num_Rooms'] != '-']
temp.head()

,Geography,Median_Num_Rooms
0,1500000US010010201001,5.2
1,1500000US010010201002,5.7
2,1500000US010010202001,4.9
3,1500000US010010202002,5.4
4,1500000US010010203001,5.7


In [284]:
# merge dfWalk with rooms data on GEOID
dfWalk = dfWalk.merge(temp, how = 'inner', left_on = 'realGEOID', right_on = 'Geography')
dfWalk = dfWalk.drop('Geography', axis = 1)
print(dfWalk.shape)
dfWalk.head()

(171306, 20)


,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,jobs_within_45_minutes_auto,jobs_within_45_minutes_transit,regional_centrality,Population,percent_non_white,Median_Contract_Rent,avg_HH_size_renters,Median_Household_Income,Median_Num_Rooms
0,1500000US481130078254,1,14.000000,Texas,0.549,0.744681,191,16.332625,7.147222,0.156028,0.003602,433601,135362,0.785893,1202,0.400998,838,2.55,54154,4.3
1,1500000US481130078253,1,8.333333,Texas,0.811,0.507599,138,27.951553,13.843035,0.000000,0.004071,404573,230587,0.733281,737,0.770692,707,2.05,33996,3.5
2,1500000US481130078271,1,6.833333,Texas,0.588,0.227364,206,26.357776,11.028441,0.006036,0.002246,402287,138562,0.729137,1336,0.561377,859,2.72,31213,4.2
3,1500000US481130093012,1,8.000000,Texas,0.644,0.915761,137,7.680394,2.252783,0.002717,0.002596,263813,8873,0.478156,1541,0.143413,581,4.81,53191,5.0
4,1500000US481130011022,1,13.166667,Texas,0.690,0.552941,302,9.110446,4.797439,0.203922,0.009142,372503,275466,0.675155,583,0.102916,1375,1.88,139875,3.1


In [285]:
# get median year built data
temp = pd.read_csv('/Users/austincoffelt/Downloads/Rent-Walk-Local/Median_Structure_2018/Median_Structure_2018.csv', header = 1)
temp = temp.rename({'Estimate!!Median year structure built --!!Renter occupied': 'Median_Year_Structure_Built'}, axis = 1)
temp = temp[['Geography', 'Median_Year_Structure_Built']]

#remove observations without numbers
temp = temp[temp['Median_Year_Structure_Built'] != '-']
temp.head()

,Geography,Median_Year_Structure_Built
0,1500000US010010201001,1965
1,1500000US010010201002,1992
2,1500000US010010202001,1976
3,1500000US010010202002,1969
4,1500000US010010203001,1975


In [286]:
# merge dfWalk with year data on GEOID
dfWalk = dfWalk.merge(temp, how = 'inner', left_on = 'realGEOID', right_on = 'Geography')
dfWalk = dfWalk.drop('Geography', axis = 1)
print(dfWalk.shape)
dfWalk.head()

(170122, 21)


,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,...,jobs_within_45_minutes_auto,jobs_within_45_minutes_transit,regional_centrality,Population,percent_non_white,Median_Contract_Rent,avg_HH_size_renters,Median_Household_Income,Median_Num_Rooms,Median_Year_Structure_Built
0,1500000US481130078254,1,14.000000,Texas,0.549,0.744681,191,16.332625,7.147222,0.156028,...,433601,135362,0.785893,1202,0.400998,838,2.55,54154,4.3,1983
1,1500000US481130078253,1,8.333333,Texas,0.811,0.507599,138,27.951553,13.843035,0.000000,...,404573,230587,0.733281,737,0.770692,707,2.05,33996,3.5,1986
2,1500000US481130078271,1,6.833333,Texas,0.588,0.227364,206,26.357776,11.028441,0.006036,...,402287,138562,0.729137,1336,0.561377,859,2.72,31213,4.2,1984
3,1500000US481130093012,1,8.000000,Texas,0.644,0.915761,137,7.680394,2.252783,0.002717,...,263813,8873,0.478156,1541,0.143413,581,4.81,53191,5.0,1957
4,1500000US481130011022,1,13.166667,Texas,0.690,0.552941,302,9.110446,4.797439,0.203922,...,372503,275466,0.675155,583,0.102916,1375,1.88,139875,3.1,1959


In [287]:
# get crime data
acCrime = pd.read_csv('Alameda_County_Violent_Crime_BG.csv')
btnCrime = pd.read_csv('Boston_Violent_Crime_BG.csv')
buffCrime = pd.read_csv('Buffalo_Violent_Crime_BG.csv')
chattCrime = pd.read_csv('Chatta_Violent_Crime_BG.csv')
cinciCrime = pd.read_csv('Cinci_Violent_Crime_BG.csv')
DetCrime = pd.read_csv('Detroit_Violent_Crime_BG.csv')
gCrime = pd.read_csv('Gaines_Violent_Crime_BG.csv')
johnCrime = pd.read_csv('J_Creek_violent_crime_by_BG.csv')
oakCrime = pd.read_csv('Oakland_Violent_Crime_BG.csv')
ralCrime = pd.read_csv('Ral_Violent_Crime_BG.csv')
ralCrime = ralCrime.rename({'AFFGEOID':'count','Unnamed: 0':'AFFGEOID'}, axis = 1) 
laCrime = pd.read_csv('LA_Violent_Crime_BG.csv')
noCrime = pd.read_csv('NO_Violent_Crime_BG.csv')
auCrime = pd.read_csv('AustinTX_violent_crime_by_BG.csv')
chiCrime = pd.read_csv('Chicago_Violent_Crime_BG.csv')
montCrime = pd.read_csv('MontMD_violent_crime_by_BG.csv')
nyCrime = pd.read_csv('NY_Violent_Crime_BG.csv')
camCrime = pd.read_csv('Cambridge_Violent_Crime_BG.csv')
huCrime = pd.read_csv('Houston_Violent_Crime_BG.csv')
sfCrime = pd.read_csv('SF_Violent_Crime_BG.csv')
crime = pd.concat([laCrime, noCrime, auCrime, chiCrime, montCrime, nyCrime, camCrime, huCrime, sfCrime, acCrime, btnCrime, 
                   buffCrime, chattCrime, cinciCrime, DetCrime, gCrime, johnCrime, oakCrime, ralCrime])
print(crime.shape)
crime.head()

(14381, 2)


,AFFGEOID,count
0,1500000US060372077101,231
1,1500000US060372260021,182
2,1500000US060372063003,162
3,1500000US060371907001,156
4,1500000US060372063001,148


In [288]:
# merge dfWalk with crime data on GEOID
dfWalk = dfWalk.merge(crime, how = 'inner', left_on = 'realGEOID', right_on = 'AFFGEOID')
dfWalk['violent_crime_rate'] = (dfWalk['count'] / dfWalk['Population']) * 100000
dfWalk = dfWalk.drop('AFFGEOID', axis = 1)
print(dfWalk.shape)
dfWalk.head()

(12123, 23)


,realGEOID,Intercept,Walk_Index,state,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,...,regional_centrality,Population,percent_non_white,Median_Contract_Rent,avg_HH_size_renters,Median_Household_Income,Median_Num_Rooms,Median_Year_Structure_Built,count,violent_crime_rate
0,1500000US481576721002,1,11.000000,Texas,0.547,0.586767,582,2.554612,8.839588,6.593009,...,0.392794,1786,0.250280,2326,1.76,131146,3.8,2014+,2,111.982083
1,1500000US483396923005,1,8.000000,Texas,0.722,0.857593,1717,3.420548,1.661340,0.500630,...,0.249754,9961,0.109828,1672,2.13,116474,4.8,2001,1,10.039153
2,1500000US484530023144,1,11.000000,Texas,0.811,0.309278,257,36.133598,13.356921,0.035346,...,0.701369,1964,0.471487,967,3.03,28495,4.1,1989,12,610.997963
3,1500000US484530002041,1,18.333333,Texas,0.822,0.344340,418,9.144881,23.882238,3.517296,...,0.953458,1161,0.192937,907,1.79,31500,2.3,1976,2,172.265289
4,1500000US484530015033,1,11.833333,Texas,0.824,0.453471,457,4.726069,21.162089,7.763663,...,0.886062,1325,0.211321,944,1.81,47078,3.7,1972,11,830.188679


In [289]:
# get dummies for the states
stateDummies = pd.get_dummies(dfWalk.loc[:, 'state'] , drop_first = False, dtype = 'int')

# concatinate with our dataframe
dfWalk = pd.concat([dfWalk, stateDummies], axis = 1)

# drop state and california for multicoliniarity issues
dfWalk = dfWalk.drop(['state', 'California'], axis = 1)
dfWalk.head()

,realGEOID,Intercept,Walk_Index,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,...,Illinois,Louisiana,Maryland,Massachusetts,Michigan,New York,North Carolina,Ohio,Tennessee,Texas
0,1500000US481576721002,1,11.000000,0.547,0.586767,582,2.554612,8.839588,6.593009,-99999.000000,...,0,0,0,0,0,0,0,0,0,1
1,1500000US483396923005,1,8.000000,0.722,0.857593,1717,3.420548,1.661340,0.500630,-99999.000000,...,0,0,0,0,0,0,0,0,0,1
2,1500000US484530023144,1,11.000000,0.811,0.309278,257,36.133598,13.356921,0.035346,0.014425,...,0,0,0,0,0,0,0,0,0,1
3,1500000US484530002041,1,18.333333,0.822,0.344340,418,9.144881,23.882238,3.517296,0.023256,...,0,0,0,0,0,0,0,0,0,1
4,1500000US484530015033,1,11.833333,0.824,0.453471,457,4.726069,21.162089,7.763663,0.034468,...,0,0,0,0,0,0,0,0,0,1


In [290]:
# remove numbers on the borders for precise measurements
dfWalk = dfWalk.loc[(dfWalk['Median_Household_Income'] != '250,000+')]
dfWalk = dfWalk.loc[(dfWalk['Median_Household_Income'] != '2,500-')]
dfWalk = dfWalk.loc[(dfWalk['Median_Contract_Rent'] != '3,500+')]
dfWalk = dfWalk.loc[(dfWalk['Median_Contract_Rent'] != '100-')]
#dfWalk.loc[(dfWalk['Median_Household_Income'] == '250,000+'), 'Median_Household_Income'] = 250000
#dfWalk.loc[(dfWalk['Median_Household_Income'] == '2,500-'), 'Median_Household_Income'] = 2500
#dfWalk.loc[(dfWalk['Median_Contract_Rent'] == '3,500+'), 'Median_Contract_Rent'] = 3500
#dfWalk.loc[(dfWalk['Median_Contract_Rent'] == '100-'), 'Median_Contract_Rent'] = 100

# log data for income and rent
dfWalk['log_Median_Household_Income'] = np.log(dfWalk['Median_Household_Income'].astype('int'))
dfWalk['log_Median_Contract_Rent'] = np.log(dfWalk['Median_Contract_Rent'].astype('int'))
dfWalk = dfWalk.drop(['realGEOID', 'Median_Household_Income', 'Median_Contract_Rent'], axis = 1)
dfWalk.head()

,Intercept,Walk_Index,percentage_work_age,percent_two_car,count_high_wage_workers,population_density,employent_housing_density,jobs_per_household,transit_frequency,jobs_within_45_minutes_auto,...,Maryland,Massachusetts,Michigan,New York,North Carolina,Ohio,Tennessee,Texas,log_Median_Household_Income,log_Median_Contract_Rent
0,1,11.000000,0.547,0.586767,582,2.554612,8.839588,6.593009,-99999.000000,214839,...,0,0,0,0,0,0,0,1,11.784066,7.751905
1,1,8.000000,0.722,0.857593,1717,3.420548,1.661340,0.500630,-99999.000000,136603,...,0,0,0,0,0,0,0,1,11.665423,7.421776
2,1,11.000000,0.811,0.309278,257,36.133598,13.356921,0.035346,0.014425,173647,...,0,0,0,0,0,0,0,1,10.257484,6.874198
3,1,18.333333,0.822,0.344340,418,9.144881,23.882238,3.517296,0.023256,236060,...,0,0,0,0,0,0,0,1,10.357743,6.810142
4,1,11.833333,0.824,0.453471,457,4.726069,21.162089,7.763663,0.034468,219374,...,0,0,0,0,0,0,0,1,10.759561,6.850126


In [291]:
# remove observations on the border change type to float
dfWalk['avg_HH_size_renters'] = dfWalk['avg_HH_size_renters'].astype('float')
dfWalk = dfWalk[(dfWalk['Median_Num_Rooms'] != '9.0+')]
#dfWalk.loc[(dfWalk['Median_Num_Rooms'] == '9.0+'), 'Median_Num_Rooms'] = 9.0
dfWalk['Median_Num_Rooms'] = dfWalk['Median_Num_Rooms'].astype('float')
dfWalk = dfWalk[dfWalk['Median_Year_Structure_Built'] != '1939-']
#dfWalk.loc[(dfWalk['Median_Year_Structure_Built'] == '1939-'), 'Median_Year_Structure_Built'] = 1939

# for 2014+ put 2016 to split the difference
dfWalk.loc[dfWalk['Median_Year_Structure_Built'] == '2014+', 'Median_Year_Structure_Built'] = 2021
dfWalk['Median_Year_Structure_Built'] = dfWalk['Median_Year_Structure_Built'].astype('int')

dfWalk.shape

(8599, 33)

In [292]:
#separate into X and y
y = dfWalk['log_Median_Contract_Rent']
X = dfWalk.drop('log_Median_Contract_Rent', axis = 1)

In [293]:
crimeQuartile = np.quantile(X['violent_crime_rate'], [0.50])

#get interaction
X['high_crime'] = 0
X.loc[(X['violent_crime_rate'] >= crimeQuartile[0]), 'high_crime'] = 1
#X['low_crime'] = 0
#X.loc[(X['violent_crime_rate'] >= crimeQuartile[0]), 'low_crime'] = 1
X['Walk_index_x_high_crime'] = X['high_crime'] * X['Walk_Index']
#X['Walk_index_x_low_crime'] = X['low_crime'] * X['Walk_Index']

#drop unneeded columns
X = X.drop(['count', 'violent_crime_rate', 'Population'], axis = 1)

In [294]:
mod = sm.OLS(y, X, axis = 1)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     log_Median_Contract_Rent   R-squared:                       0.732
Model:                                  OLS   Adj. R-squared:                  0.731
Method:                       Least Squares   F-statistic:                     779.1
Date:                      Wed, 23 Apr 2025   Prob (F-statistic):               0.00
Time:                              00:45:06   Log-Likelihood:                 99.179
No. Observations:                      8599   AIC:                            -136.4
Df Residuals:                          8568   BIC:                             82.48
Df Model:                                30                                         
Covariance Type:                  nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         -2.7162      0.379     -7.174      0.000      -3.458      -1.974
Walk_Index                         0.0084      0.001      5.793      0.000       0.006       0.011
percentage_work_age                0.5069      0.030     16.642      0.000       0.447       0.567
percent_two_car                   -0.3024      0.022    -13.615      0.000      -0.346      -0.259
count_high_wage_workers         7.435e-05   1.23e-05      6.045      0.000    5.02e-05    9.85e-05
population_density                1.9e-05   7.34e-05      0.259      0.796      -0.000       0.000
employent_housing_density          0.0003   4.08e-05      8.042      0.000       0.000       0.000
jobs_per_household              5.277e-06      0.000      0.043      0.966      -0.000       0.000
transit_frequency              -1.318e-07   1.79e-07     -0.737      0.461   -4.82e-07    2.19e-07
jobs_within_45_minutes_auto     2.607e-07   2.81e-08      9.281      0.000    2.06e-07    3.16e-07
jobs_within_45_minutes_transit  5.685e-09   5.69e-09      1.000      0.318   -5.46e-09    1.68e-08
regional_centrality               -0.2174      0.025     -8.761      0.000      -0.266      -0.169
percent_non_white                 -0.0836      0.012     -6.998      0.000      -0.107      -0.060
avg_HH_size_renters                0.0272      0.004      7.479      0.000       0.020       0.034
Median_Num_Rooms                   0.0562      0.004     14.778      0.000       0.049       0.064
Median_Year_Structure_Built        0.0021      0.000     11.038      0.000       0.002       0.002
Florida                           -0.2413      0.035     -6.830      0.000      -0.310      -0.172
Georgia                           -0.2884      0.071     -4.081      0.000      -0.427      -0.150
Illinois                          -0.3409      0.011    -32.190      0.000      -0.362      -0.320
Louisiana                         -0.1771      0.020     -8.874      0.000      -0.216      -0.138
Maryland                          -0.0990      0.016     -6.293      0.000      -0.130      -0.068
Massachusetts                     -0.0816      0.019     -4.395      0.000      -0.118      -0.045
Michigan                          -0.3813      0.015    -24.816      0.000      -0.411      -0.351
New York                          -0.2335      0.013    -17.521      0.000      -0.260      -0.207
North Carolina                    -0.3212      0.022    -14.548      0.000      -0.364      -0.278
Ohio                              -0.4733      0.022    -21.274      0.000      -0.517      -0.430
Tennessee                         -0.4099      0.027    -15.335      0.000      -0.462      -0.357
Texas                             -0.3369      

In [295]:
robust_results = res.get_robustcov_results(cov_type='HC1')
robust_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               OLS Regression Results                               
====================================================================================
Dep. Variable:     log_Median_Contract_Rent   R-squared:                       0.732
Model:                                  OLS   Adj. R-squared:                  0.731
Method:                       Least Squares   F-statistic:                     767.8
Date:                      Wed, 23 Apr 2025   Prob (F-statistic):               0.00
Time:                              00:45:06   Log-Likelihood:                 99.179
No. Observations:                      8599   AIC:                            -136.4
Df Residuals:                          8568   BIC:                             82.48
Df Model:                                30                                         
Covariance Type:                        HC1                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         -2.7162      0.426     -6.380      0.000      -3.551      -1.882
Walk_Index                         0.0084      0.002      5.556      0.000       0.005       0.011
percentage_work_age                0.5069      0.039     12.971      0.000       0.430       0.584
percent_two_car                   -0.3024      0.025    -12.305      0.000      -0.351      -0.254
count_high_wage_workers         7.435e-05   1.41e-05      5.288      0.000    4.68e-05       0.000
population_density                1.9e-05   8.41e-05      0.226      0.821      -0.000       0.000
employent_housing_density          0.0003   4.57e-05      7.175      0.000       0.000       0.000
jobs_per_household              5.277e-06      0.000      0.026      0.979      -0.000       0.000
transit_frequency              -1.318e-07   1.58e-07     -0.833      0.405   -4.42e-07    1.78e-07
jobs_within_45_minutes_auto     2.607e-07   3.24e-08      8.044      0.000    1.97e-07    3.24e-07
jobs_within_45_minutes_transit  5.685e-09   6.07e-09      0.936      0.349   -6.22e-09    1.76e-08
regional_centrality               -0.2174      0.028     -7.829      0.000      -0.272      -0.163
percent_non_white                 -0.0836      0.012     -6.882      0.000      -0.107      -0.060
avg_HH_size_renters                0.0272      0.004      6.831      0.000       0.019       0.035
Median_Num_Rooms                   0.0562      0.004     12.761      0.000       0.048       0.065
Median_Year_Structure_Built        0.0021      0.000      9.737      0.000       0.002       0.002
Florida                           -0.2413      0.037     -6.452      0.000      -0.315      -0.168
Georgia                           -0.2884      0.046     -6.319      0.000      -0.378      -0.199
Illinois                          -0.3409      0.011    -31.626      0.000      -0.362      -0.320
Louisiana                         -0.1771      0.021     -8.384      0.000      -0.218      -0.136
Maryland                          -0.0990      0.018     -5.585      0.000      -0.134      -0.064
Massachusetts                     -0.0816      0.026     -3.157      0.002      -0.132      -0.031
Michigan                          -0.3813      0.016    -23.123      0.000      -0.414      -0.349
New York                          -0.2335      0.015    -15.565      0.000      -0.263      -0.204
North Carolina                    -0.3212      0.019    -17.025      0.000      -0.358      -0.284
Ohio                              -0.4733      0.024    -19.603      0.000      -0.521      -0.426
Tennessee                         -0.4099      0.024    -17.439      0.000      -0.456      -0.364
Texas                             -0.3369      